In [1]:
import pandas as pd
import os

In [2]:
df = pd.DataFrame()
path = "data/"
directory = os.fsencode(path)

for file in os.listdir(directory):
    filename = os.fsdecode(file)
    if filename.endswith(".csv") and filename.startswith("ntee_codes")==False: 
        print("Loading file: " + filename)
        temp = pd.read_csv(path+filename,usecols=['EIN','NTEE_CD'])
        df = df.append(temp, ignore_index = True) 


Loading file: eo1.csv
Loading file: eo3.csv
Loading file: eo2.csv
Loading file: eo4.csv


In [7]:
df.rename(columns = {"NTEE_CD":'code'},inplace=True) # for a later join

ntee = pd.read_csv('data/ntee_codes.csv')
## Left join on 'code' column
datamerge = pd.merge(df, ntee, on='code',how='inner')
datamerge['EIN'] = datamerge['EIN'].apply(lambda x: str(x).zfill(9))
datamerge.head()

,EIN,code,description,category
0,000587764,X21,Protestant,religious
1,000635913,X21,Protestant,religious
2,007764840,X21,Protestant,religious
3,010347153,X21,Protestant,religious
4,010348158,X21,Protestant,religious


In [8]:
%run ../api/classy990downloads.ipynb


year = 2017
uri='mongodb://mongo/irs990'

db = mongo990( uri=uri, year=year, dump=False)
        
db.verify()

importing Jupyter notebook from /home/notebooks/api/filter_text.ipynb


[nltk_data] Downloading package wordnet to /home/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


importing Jupyter notebook from /home/notebooks/api/ParseWeb.ipynb
Connect to: [mongodb://mongo/irs990]
List of databases in MongoDB: ['admin', 'config', 'irs990', 'local']
Open db: [irs990]
Collection: [2017]
1170 documents present
1170 documents present


[nltk_data] Downloading package punkt to /home/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /home/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [10]:
cursor = db.collection.find( {},{ 'EIN':1 } )
mongo_data =  pd.DataFrame(list(cursor))
mongo_data.head()

,EIN,_id
0,042662873,042662873_201603
1,042964630,042964630_201512
2,382912028,382912028_201512
3,200509226,200509226_201605
4,202699020,202699020_201512


In [11]:
allmerge = pd.merge(datamerge, mongo_data, on='EIN',how='inner')
allmerge.head()

,EIN,code,description,category,_id
0,320136834,X21,Protestant,religious,320136834_201512
1,311787775,X21,Protestant,religious,311787775_201512
2,364070694,X21,Protestant,religious,364070694_201508
3,202058546,X20,Christianity,religious,202058546_201512
4,208493868,X20,Christianity,religious,208493868_201512


In [12]:
cll = db.collection
bulk_update = cll.initialize_unordered_bulk_op()
for index, row in allmerge.iterrows():
    EIN=str(row['EIN'])
    code=row['code']
    desc = row['description']
    category = row['category']
    bulk_update.find({"EIN":EIN}).update_one({"$set":{'NTEECode':code,'NTEEDesc':desc,'NTEECategory':category}})
bulk_update.execute()

/usr/local/lib/python3.5/dist-packages/ipykernel_launcher.py:2: DeprecationWarning: initialize_unordered_bulk_op is deprecated
  


{'nInserted': 0,
 'nMatched': 584,
 'nModified': 584,
 'nRemoved': 0,
 'nUpserted': 0,
 'upserted': [],
 'writeConcernErrors': [],
 'writeErrors': []}